In [1]:
import pyspark.sql.functions as f
from graphframes import GraphFrame
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, IntegerType, StringType, DoubleType

In [2]:
spark = SparkSession.builder.appName("YoutubeANalyser").getOrCreate()

In [3]:
schema = StructType() \
    .add("video_id", StringType(), True) \
    .add("uploader", StringType(), True) \
    .add("age", IntegerType(), True) \
    .add("category", StringType(), True) \
    .add("length", IntegerType(), True) \
    .add("views", IntegerType(), True) \
    .add("rate", DoubleType(), True) \
    .add("ratings", IntegerType(), True) \
    .add("comments", IntegerType(), True) \
    .add("ref1", StringType(), True) \
    .add("ref2", StringType(), True) \
    .add("ref3", StringType(), True) \
    .add("ref4", StringType(), True) \
    .add("ref5", StringType(), True) \
    .add("ref6", StringType(), True) \
    .add("ref7", StringType(), True) \
    .add("ref8", StringType(), True) \
    .add("ref9", StringType(), True) \
    .add("ref10", StringType(), True) \
    .add("ref11", StringType(), True) \
    .add("ref12", StringType(), True) \
    .add("ref13", StringType(), True) \
    .add("ref14", StringType(), True) \
    .add("ref15", StringType(), True) \
    .add("ref16", StringType(), True) \
    .add("ref17", StringType(), True) \
    .add("ref18", StringType(), True) \
    .add("ref19", StringType(), True) \
    .add("ref20", StringType(), True)


In [4]:
df = spark.read.format("csv") \
    .option("header", True) \
    .option("delimiter", "\t") \
    .schema(schema) \
    .option("inferSchema", "true") \
    .load("1.txt")

In [5]:
video_length = StructType() \
    .add("id", StringType(), True) \
    .add("length", IntegerType(), True)

In [6]:
video_size = StructType() \
    .add("id_", StringType(), True) \
    .add("length_", IntegerType(), True) \
    .add("size", IntegerType(), True)

In [7]:
dfVideoLength = spark.read.format("csv") \
    .option("header", True) \
    .option("delimiter", "\t") \
    .schema(video_length) \
    .option("inferSchema", "true") \
    .load("idlength.txt")

In [8]:
dfVideoSize = spark.read.format("csv") \
    .option("header", True) \
    .option("delimiter", "\t") \
    .schema(video_size) \
    .option("inferSchema", "true") \
    .load("size.txt")

In [9]:
dfMappedLength = dfVideoLength.join(dfVideoSize, dfVideoSize.id_ == dfVideoLength.id).select('id', 'length_', 'size')


In [10]:
dfRawData = df.select("video_id", "uploader", "age", "category", "length", "views", "rate", "ratings", "comments")


In [11]:
dfSIzeMappedWithRaw = dfMappedLength.join(dfRawData, dfRawData.video_id == dfMappedLength.id).select('*')
dfSIzeMappedWithRaw.show()

+-----------+-------+--------+-----------+----------------+---+----------------+------+------+----+-------+--------+
|         id|length_|    size|   video_id|        uploader|age|        category|length| views|rate|ratings|comments|
+-----------+-------+--------+-----------+----------------+---+----------------+------+------+----+-------+--------+
|vy74CnRaQgs|    420|17390492|vy74CnRaQgs|       supreme84|697|          Sports|   420| 35186|4.68|     44|      51|
|6hJCfXllrqE|    309|13497109|6hJCfXllrqE|  DanielSundgren|613|           Music|   309|  1230|2.67|      3|       0|
|GvIFh3por14|    538|22236234|GvIFh3por14|   CharlesCoburn|690|   Entertainment|   538| 12788|4.56|     25|      41|
|Eykcs8v77N0|      9|  352917|Eykcs8v77N0|        gweebage|600|  People & Blogs|     9| 12176|3.67|      9|       0|
|LqiLaTpIhD4|    149| 5994499|LqiLaTpIhD4| MyCadillacStory|722|Autos & Vehicles|   149|  5178| 4.2|     20|      16|
|uObTAvVz2iY|     27| 1005034|uObTAvVz2iY| MyCadillacStory|716|A

In [12]:
columns = [f.col('ref1'), f.col('ref2'), f.col('ref3'), f.col('ref4'), f.col('ref5'), f.col('ref6'), f.col('ref7'),
           f.col('ref8'), f.col('ref9'), f.col('ref10'), f.col('ref11'), f.col('ref12'), f.col('ref13'), f.col('ref14'),
           f.col('ref15'), f.col('ref16'), f.col('ref17'), f.col('ref18'), f.col('ref19'), f.col('ref20')]

In [13]:
output = df.withColumn("related", f.array(columns)).select("video_id", "related")

In [40]:
output

DataFrame[video_id: string, related: array<string>]

In [41]:
output.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- related: array (nullable = false)
 |    |-- element: string (containsNull = true)



In [14]:
dfEdge = output.select(output.video_id, f.explode(output.related))
dfEdge = dfEdge.withColumnRenamed("col", "dst") \
    .withColumnRenamed("video_id", "src")

In [22]:
g = GraphFrame(dfSIzeMappedWithRaw, dfEdge)

In [23]:
g.inDegrees.show()

+-----------+--------+
|         id|inDegree|
+-----------+--------+
|71agSyF_TYo|       4|
|MydHXSjZgp4|      19|
|CVqSmBSEe0I|       1|
|z-fQEgmQFXE|       1|
|NBSQWuujn-I|       1|
|82nR5p-XDs8|       2|
|OdiwzztQtTE|       1|
|pSnaNtvfTqg|       1|
|grZIe1wKEH4|       1|
|9iS97D0Rkko|       2|
|8vuIsp5J-bU|       1|
|zopZmUhKduI|       1|
|OKiXkQYatfA|       1|
|JaOoaE8h7n8|       1|
|-6TUKuLXaJU|       1|
|SxxBsNbdhhY|       1|
|Sr5p09qkG34|       1|
|gd2qoz5eL8k|       6|
|BCQvVTGng28|       1|
|bhVvC3zuvSY|       2|
+-----------+--------+
only showing top 20 rows



In [24]:
# MAX indegree
g.inDegrees.agg({"inDegree": "max"}).show()

+-------------+
|max(inDegree)|
+-------------+
|         2906|
+-------------+



In [25]:
# MIN indegree
g.inDegrees.agg({"inDegree": "min"}).show()

+-------------+
|min(inDegree)|
+-------------+
|            1|
+-------------+



In [26]:
# AVG indegree
g.inDegrees.agg({"inDegree": "avg"}).show()

+------------------+
|     avg(inDegree)|
+------------------+
|2.1884406983744733|
+------------------+



In [27]:
g.outDegrees.show()

+-----------+---------+
|         id|outDegree|
+-----------+---------+
|MydHXSjZgp4|       20|
|WkY4gKaPNBw|       20|
|gd2qoz5eL8k|       20|
|_9fZtJsq1gI|       20|
|jKxg40VySI4|       20|
|ZyWgo4QMrfQ|       20|
|zMSpTvv9tg8|       20|
|2o3lFsf3fkQ|       20|
|3866CS6QVC8|       20|
|8XrLxo15i48|       20|
|hmh1xt7yiVs|       20|
|_hDsHDjERA0|       20|
|rQDqbpq3lpU|       20|
|yUWD2vZbO84|       20|
|GcIEpyem6XY|       20|
|ZqF5_OD-4Xw|       20|
|0Kc4_YpOHlM|       20|
|dATGTZm3Ypc|       20|
|7ZCjnSkpyZc|       20|
|ntuFXBvwIl8|       20|
+-----------+---------+
only showing top 20 rows



In [28]:
# MAX outdegree
g.outDegrees.agg({"outDegree": "max"}).show()

+--------------+
|max(outDegree)|
+--------------+
|            20|
+--------------+



In [29]:
# MIN outdegree
g.outDegrees.agg({"outDegree": "min"}).show()

+--------------+
|min(outDegree)|
+--------------+
|            20|
+--------------+



In [30]:
# AVG outdegree
g.outDegrees.agg({"outDegree": "avg"}).show()

+--------------+
|avg(outDegree)|
+--------------+
|          20.0|
+--------------+



In [31]:
# B. Search
# - top k queries
g.vertices.groupBy("category").count().sort(f.col("count").desc()).show()

+----------------+-----+
|        category|count|
+----------------+-----+
|           Music|   22|
|Autos & Vehicles|   17|
| News & Politics|   14|
|     Howto & DIY|   10|
|  People & Blogs|    9|
|   Entertainment|    7|
|          Sports|    7|
| Gadgets & Games|    6|
|          Comedy|    5|
|            UNA |    2|
|Film & Animation|    2|
| Travel & Places|    1|
+----------------+-----+



In [32]:
# Range queries: find all videos in categories X with duration within a range [t1, t2]; find all
g.vertices.filter((f.col('length').between(0, 9999999999)) & (f.col('category') == 'Sports')).show()

+-----------+-------+--------+-----------+-----------+---+--------+------+-----+----+-------+--------+
|         id|length_|    size|   video_id|   uploader|age|category|length|views|rate|ratings|comments|
+-----------+-------+--------+-----------+-----------+---+--------+------+-----+----+-------+--------+
|vy74CnRaQgs|    420|17390492|vy74CnRaQgs|  supreme84|697|  Sports|   420|35186|4.68|     44|      51|
|3qNVLn0rNYk|    311|12966459|3qNVLn0rNYk|     jhsieh|502|  Sports|   311|10683|4.53|     15|       3|
|8CCSF0G84HQ|    219| 7661791|8CCSF0G84HQ|  truckitup|517|  Sports|   219| 2276|4.33|      3|       0|
|vyiVL0zPpaE|    254|10490566|vyiVL0zPpaE|glidinclyde|457|  Sports|   254| 2459| 5.0|      8|       7|
|S1imSSkqOB8|    242| 8730888|S1imSSkqOB8|    leasky8|671|  Sports|   242| 4113|4.35|     17|      15|
|28gYtVPOm5U|    216| 8953391|28gYtVPOm5U| xlcpuvirus|646|  Sports|   216| 4019|4.62|     16|       2|
|KzbYR_Ri3ww|     96| 3708444|KzbYR_Ri3ww|     Jlinck|658|  Sports|    96

In [34]:
# videos with size in range [x,y].
g.vertices.groupBy("uploader").count().sort(f.col("count").desc()).show()

+---------------+-----+
|       uploader|count|
+---------------+-----+
|MyCadillacStory|   11|
|     lilscrappy|    9|
|       heniadir|    3|
|  bigwallypants|    2|
|        thirdd3|    2|
|          reeuh|    1|
|      truckitup|    1|
|       yatucamp|    1|
|     xlcpuvirus|    1|
|        jensyao|    1|
|       kylaalee|    1|
|     bishop8000|    1|
|    glidinclyde|    1|
|       gweebage|    1|
|     liamtipton|    1|
| DanielSundgren|    1|
|  coyoteboy1983|    1|
|     YagoCooper|    1|
|          VVasp|    1|
|        erkut89|    1|
+---------------+-----+
only showing top 20 rows



In [35]:
g.vertices.filter((f.col('size').between(0, 9999999999)) & (f.col('category') == 'Sports')).show()

+-----------+-------+--------+-----------+-----------+---+--------+------+-----+----+-------+--------+
|         id|length_|    size|   video_id|   uploader|age|category|length|views|rate|ratings|comments|
+-----------+-------+--------+-----------+-----------+---+--------+------+-----+----+-------+--------+
|vy74CnRaQgs|    420|17390492|vy74CnRaQgs|  supreme84|697|  Sports|   420|35186|4.68|     44|      51|
|3qNVLn0rNYk|    311|12966459|3qNVLn0rNYk|     jhsieh|502|  Sports|   311|10683|4.53|     15|       3|
|8CCSF0G84HQ|    219| 7661791|8CCSF0G84HQ|  truckitup|517|  Sports|   219| 2276|4.33|      3|       0|
|vyiVL0zPpaE|    254|10490566|vyiVL0zPpaE|glidinclyde|457|  Sports|   254| 2459| 5.0|      8|       7|
|S1imSSkqOB8|    242| 8730888|S1imSSkqOB8|    leasky8|671|  Sports|   242| 4113|4.35|     17|      15|
|28gYtVPOm5U|    216| 8953391|28gYtVPOm5U| xlcpuvirus|646|  Sports|   216| 4019|4.62|     16|       2|
|KzbYR_Ri3ww|     96| 3708444|KzbYR_Ri3ww|     Jlinck|658|  Sports|    96

In [36]:
# C. PageRank
pr = g.pageRank(resetProbability=0.15, tol=0.01)
pr.vertices.show()

+-----------+-------+--------+-----------+----------------+---+----------------+------+------+----+-------+--------+------------------+
|         id|length_|    size|   video_id|        uploader|age|        category|length| views|rate|ratings|comments|          pagerank|
+-----------+-------+--------+-----------+----------------+---+----------------+------+------+----+-------+--------+------------------+
|vy74CnRaQgs|    420|17390492|vy74CnRaQgs|       supreme84|697|          Sports|   420| 35186|4.68|     44|      51|0.4049279991415213|
|6hJCfXllrqE|    309|13497109|6hJCfXllrqE|  DanielSundgren|613|           Music|   309|  1230|2.67|      3|       0|0.4049279991415213|
|GvIFh3por14|    538|22236234|GvIFh3por14|   CharlesCoburn|690|   Entertainment|   538| 12788|4.56|     25|      41|0.4049279991415213|
|Eykcs8v77N0|      9|  352917|Eykcs8v77N0|        gweebage|600|  People & Blogs|     9| 12176|3.67|      9|       0|0.4049279991415213|
|LqiLaTpIhD4|    149| 5994499|LqiLaTpIhD4| MyCad

In [37]:
pr.edges.show()

+-----------+-----------+-------------------+
|        src|        dst|             weight|
+-----------+-----------+-------------------+
|TFWNpS4E4fM|gSP9EnBHI10|               0.25|
|BU7L9OawuMI|gSP9EnBHI10|               0.25|
|q8u2TSrw_Pc|r76T2EG90MA|                0.2|
|_yUo9crgm3U|r76T2EG90MA|0.16666666666666666|
|LqiLaTpIhD4|6qc9bMuffoE|0.09090909090909091|
|uObTAvVz2iY|6qc9bMuffoE|0.09090909090909091|
|Q6KmM2_jsR0|6qc9bMuffoE|0.09090909090909091|
|6qc9bMuffoE|6qc9bMuffoE|0.09090909090909091|
|0ZH7NBiruqk|6qc9bMuffoE|0.09090909090909091|
|CycZagvqJps|6qc9bMuffoE|0.09090909090909091|
|g9m-Mjk9a44|6qc9bMuffoE|0.09090909090909091|
|CMNaxTCbgJU|6qc9bMuffoE|0.09090909090909091|
|qsrnHkeKtqs|6qc9bMuffoE|0.09090909090909091|
|vXZEPt7mbgY|6qc9bMuffoE|0.09090909090909091|
|v28pkP3WCLw|6qc9bMuffoE|0.09090909090909091|
|AQXj7ANHP-Q|40NRsg0UsA0| 0.1111111111111111|
|Hyfmg9zrbYI|40NRsg0UsA0| 0.1111111111111111|
|40NRsg0UsA0|40NRsg0UsA0| 0.1111111111111111|
|abkj32lgDKc|40NRsg0UsA0| 0.111111